In [58]:
import pandas as pd
import numpy as np
import csv as csv
import matplotlib.pyplot as plt
%matplotlib inline 
import keras
from keras.utils import np_utils
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier 
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.datasets import make_classification

#pd.set_option('display.max_colwidth', -1)
#pd.set_option('display.max_columns', 500)

In [59]:
df = pd.read_csv('algebra_2005_2006_train.txt', sep='\t')

The following code splits the KC(Default) and Opportunity(Default) variables from strings into lists of strings.

In [60]:
df['KC(Default)'] = df['KC(Default)'].str.split('~~') #split string into list
df['Opportunity(Default)'] = df['Opportunity(Default)'].str.split('~~') #split string into list

This code creates a dictionary for each transaction mapping each KC value to the corresponding OC value. The list of dictionaries is temporarily appended to the dataframe to be used in exploding the dataframe in the next step. NaN values for KCs are replaced with empty dictionaries to be filled in with zero values later. 

In [61]:
head = df[['Row', 'KC(Default)', 'Opportunity(Default)']]
d = []
for index, row in head.iterrows():
    if np.all(pd.isnull(row['KC(Default)'])):
        d.append({})
    else:
        keys = row['KC(Default)']
        values = row['Opportunity(Default)']
        dictionary = dict(zip(keys, values))
        d.append(dictionary)
s = pd.Series(d)

In [62]:
s = s.to_frame()
head = head.join(s)
head.rename(columns = {0:'nested'}, inplace = True)
head = head.drop('KC(Default)', 1)
head = head.drop('Opportunity(Default)', 1)

This unpacking function creates a column for each KC variable and puts the corresponding OC number for the transaction in the row. If the KC is not present in the transaction, the row value for that column is 0. 

In [63]:
def unpack(df, column):
    ret = pd.concat([df, pd.DataFrame((d for idx, d in df[column].iteritems())).fillna(0)], axis=1)
    ret = ret.drop(column, 1)
    return ret

head = head.applymap(lambda x: {} if pd.isnull(x) else x)
df_unpacked = unpack(head, 'nested')
df_unpacked = df_unpacked.drop('Row', 1)

Here we are shortening the KC column names into more readable and standard names.

In [64]:
kc = ['Changing axis bounds',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
'Changing axis intervals',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             
'Choose Graphical Refl-v',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             
'Choose Graphical a',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
'Choose Graphical h',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
'Choose Graphical k',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
'Convert unit, mixed',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 
'Convert unit, multiplier',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
'Convert unit, standard',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              
'Correctly placing points',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
'Define Variable',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     
'Edit Algebraic a',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    
'Edit Algebraic h',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    
'Edit Algebraic k',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    
'Edit Algebraic refl-v',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               
'Entering a computed linear value',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    
'Entering a computed quadratic value  ',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               
'Entering a given',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    
'Entering a given linear value',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
'Entering a given quadratic value',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    
'Entering a point',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    
'Entering slope, GLF',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 
'Entering slope, SIF',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 
'Entering the slope',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
'Entering the y-intercept',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
'Entering x-intercept, GLF',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           
'Entering x-intercept, SIF',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           
'Entering y-intercept, GLF',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           
'Entering y-intercept, SIF',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           
'Excluding the line when shading',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     
'Find X, Simple',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      
'Find X, negative slope',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              
'Find X, positive slope',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              
'Find Y, Simple',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     
'Find Y, any form',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    
'Find Y, negative slope',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              
'Find Y, positive slope',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              
'Identify Parent Curve',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               
'Identify Parent Description',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         
'Identify Parent Equation',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
'Identifying units',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
'Including the line when shading',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     
'Labelling point of intersection',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     
'Labelling the axes',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
'Negative Constant, GLF',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              
'Negative Constant, SIF',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              
'Placing coordinate point',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
'Positive Constants, GLF',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             
'Positive Constants, SIF',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             
'Setting the slope',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
'Setting the y-intercept',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             
'Shading GLF equation with negative slope',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
'Shading GLF equation with positive slope',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
'Shading SIF equation with negative slope',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
'Shading SIF equation with positive slope',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
'Shading greater than',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
'Shading less than',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
'Using difficult numbers',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             
'Using large numbers',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 
'Using simple numbers',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
'Using small numbers',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 
'Write expression, initial and change',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
'Write expression, initial and point',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 
'Write expression, negative slope',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    
'Write expression, positive slope',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    
'Write expression, quadratic',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         
'Write expression, ratio',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             
'Write expression, two points',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
'SkillRule: Add/Subtract',                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
'SkillRule: Apply Exponent',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
'SkillRule: Calculate Eliminate Parens',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         
'SkillRule: Calculate negative coefficient',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              
'SkillRule: Consolidate vars with coeff',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
'SkillRule: Consolidate vars, any',
'SkillRule: Consolidate vars, no coeff',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      
'SkillRule: Do Multiply - Whole nested',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
'SkillRule: Done?',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
'SkillRule: Eliminate Parens 1',
'SkillRule: Eliminate Parens 2',                                                                                                                                                                                                                                                                                                                                                                                     
'SkillRule: Extract to consolidate vars',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               
'SkillRule: Isolate negative',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     
'SkillRule: Isolate positive',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     
'SkillRule: Make variable positive 1',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             
'SkillRule: Make variable positive 2',                                                                                                                                                                                                   
'SkillRule: Multiply/Divide',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               
'SkillRule: Remove coefficient',
'SkillRule: Remove constant',                                                                                                                                       
'SkillRule: Remove negative coefficient',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               
'SkillRule: Remove positive coefficient',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               
'SkillRule: Select Combine Terms',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              
'SkillRule: Select Eliminate Parens',  
'SkillRule: Select Multiply/Divide, nested',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          
'SkillRule: Select Multiply',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     
'SkillRule: Variable in denominator',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
'SkillRule: ax+b=c, negative',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
'SkillRule: done infinite solutions',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      
'SkillRule: done no solutions',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
'SkillRule: invert-mult',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
'combine-like-terms-r-sp',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             
'combine-like-terms-sp',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               
'combine-like-terms-whole-sp',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         
'distribute-sp',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
'factor-quadratic-sp',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 
'factor-sp',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           
'perform-mult-r-sp',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
'perform-mult-row2-sp',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
'perform-mult-sp',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     
'perform-mult-whole-sp',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               
'qft-den-sp',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          
'qft-num1-sp',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         
'qft-num2-sp',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         
'simplify-fractions-sp',]

In [65]:
df_kc = df_unpacked.columns.tolist()
d = dict(zip(df_kc, kc))
df_unpacked = df_unpacked.rename(columns = d)

We are calculating the Sum_OCs and 'Num_KCs' features.

In [66]:
df_unpacked['Correct First Attempt'] = df['Correct First Attempt']
df_unpacked['Sum_OCs'] = (df_unpacked[df_unpacked.columns[:112]].astype(int)).sum(axis=1)
df_unpacked['Num_KCs'] = (df_unpacked[df_unpacked.columns[:112]].astype(int) > 0).sum(axis=1)

We are encoding the Problem Name and Problem Hierarchy into categories based on substrings found in the variable names. This allows us to capture some information from the variable names without including too many columns by encoding each unique Problem Name/Problem Hierarchy value separately.

In [67]:
df['EG'] = df['Problem Name'].map(lambda s: 1 if 'EG' in s else 0)
df['DEMO'] = df['Problem Name'].map(lambda s: 1 if 'DEMO' in s else 0)
df['JAN'] = df['Problem Name'].map(lambda s: 1 if 'JAN' in s else 0)
df['FEB'] = df['Problem Name'].map(lambda s: 1 if 'FEB' in s else 0)
df['TRANSFORM'] = df['Problem Name'].map(lambda s: 1 if 'TRANSFORM' in s else 0)
df['LIT'] = df['Problem Name'].map(lambda s: 1 if 'LIT' in s else 0)
df['SYS'] = df['Problem Name'].map(lambda s: 1 if 'SYS' in s else 0)
df['BH'] = df['Problem Name'].map(lambda s: 1 if 'BH' in s else 0)
df['DIST'] = df['Problem Name'].map(lambda s: 1 if 'DIST' in s else 0)
df['NEW'] = df['Problem Name'].map(lambda s: 1 if 'NEW' in s else 0)
df['L5'] = df['Problem Name'].map(lambda s: 1 if 'L5' in s else 0)
df['L6'] = df['Problem Name'].map(lambda s: 1 if 'L6' in s else 0)
df['DEC'] = df['Problem Name'].map(lambda s: 1 if 'DEC' in s else 0)
df['REAL'] = df['Problem Name'].map(lambda s: 1 if 'REAL' in s else 0)
df['FOR'] = df['Problem Name'].map(lambda s: 1 if 'FOR' in s else 0)
df['1PT'] = df['Problem Name'].map(lambda s: 1 if '1PT' in s else 0)
df['2PT'] = df['Problem Name'].map(lambda s: 1 if '2PT' in s else 0)
df['PROP'] = df['Problem Name'].map(lambda s: 1 if 'PROP' in s else 0)
df['L1F'] = df['Problem Name'].map(lambda s: 1 if 'L1F' in s else 0)
df['NOV'] = df['Problem Name'].map(lambda s: 1 if 'NOV' in s else 0)
df['PERCENT'] = df['Problem Name'].map(lambda s: 1 if 'PERCENT' in s else 0)
df['IPC'] = df['Problem Name'].map(lambda s: 1 if 'IPC' in s else 0)
df['CTA'] = df['Problem Hierarchy'].map(lambda s: 1 if 'CTA' in s else 0)
df['ES'] = df['Problem Hierarchy'].map(lambda s: 1 if 'ES' in s else 0)
df['LINEAR'] = df['Problem Hierarchy'].map(lambda s: 1 if 'LINEAR' in s else 0)
df['QUAD'] = df['Problem Hierarchy'].map(lambda s: 1 if ('QUAD' in s or 'Quad' in s) else 0)

In [68]:
df_unpacked = df_unpacked.join(df[df.columns[19:]])
df_unpacked = df_unpacked.join(df['Problem View'])

In [69]:
df_unpacked.head(5)

,Changing axis bounds,Changing axis intervals,Choose Graphical Refl-v,Choose Graphical a,Choose Graphical h,Choose Graphical k,"Convert unit, mixed","Convert unit, multiplier","Convert unit, standard",Correctly placing points,Define Variable,Edit Algebraic a,Edit Algebraic h,Edit Algebraic k,Edit Algebraic refl-v,Entering a computed linear value,Entering a computed quadratic value,Entering a given,Entering a given linear value,Entering a given quadratic value,Entering a point,"Entering slope, GLF","Entering slope, SIF",Entering the slope,Entering the y-intercept,"Entering x-intercept, GLF","Entering x-intercept, SIF","Entering y-intercept, GLF","Entering y-intercept, SIF",Excluding the line when shading,"Find X, Simple","Find X, negative slope","Find X, positive slope","Find Y, Simple","Find Y, any form","Find Y, negative slope","Find Y, positive slope",Identify Parent Curve,Identify Parent Description,Identify Parent Equation,Identifying units,Including the line when shading,Labelling point of intersection,Labelling the axes,"Negative Constant, GLF","Negative Constant, SIF",Placing coordinate point,"Positive Constants, GLF","Positive Constants, SIF",Setting the slope,Setting the y-intercept,Shading GLF equation with negative slope,Shading GLF equation with positive slope,Shading SIF equation with negative slope,Shading SIF equation with positive slope,Shading greater than,Shading less than,Using difficult numbers,Using large numbers,Using simple numbers,Using small numbers,"Write expression, initial and change","Write expression, initial and point","Write expression, negative slope","Write expression, positive slope","Write expression, quadratic","Write expression, ratio","Write expression, two points",SkillRule: Add/Subtract,SkillRule: Apply Exponent,SkillRule: Calculate Eliminate Parens,SkillRule: Calculate negative coefficient,SkillRule: Consolidate vars with coeff,"SkillRule: Consolidate vars, any","SkillRule: Consolidate vars, no coeff",SkillRule: Do Multiply - Whole nested,SkillRule: Done?,SkillRule: Eliminate Parens 1,SkillRule: Eliminate Parens 2,SkillRule: Extract to consolidate vars,SkillRule: Isolate negative,SkillRule: Isolate positive,SkillRule: Make variable positive 1,SkillRule: Make variable positive 2,SkillRule: Multiply/Divide,SkillRule: Remove coefficient,SkillRule: Remove constant,SkillRule: Remove negative coefficient,SkillRule: Remove positive coefficient,SkillRule: Select Combine Terms,SkillRule: Select Eliminate Parens,"SkillRule: Select Multiply/Divide, nested",SkillRule: Select Multiply,SkillRule: Variable in denominator,"SkillRule: ax+b=c, negative",SkillRule: done infinite solutions,SkillRule: done no solutions,SkillRule: invert-mult,combine-like-terms-r-sp,combine-like-terms-sp,combine-like-terms-whole-sp,distribute-sp,factor-quadratic-sp,factor-sp,perform-mult-r-sp,perform-mult-row2-sp,perform-mult-sp,perform-mult-whole-sp,qft-den-sp,qft-num1-sp,qft-num2-sp,simplify-fractions-sp,Correct First Attempt,Sum_OCs,Num_KCs,EG,DEMO,JAN,FEB,TRANSFORM,LIT,SYS,BH,DIST,NEW,L5,L6,DEC,REAL,FOR,1PT,2PT,PROP,L1F,NOV,PERCENT,IPC,CTA,ES,LINEAR,QUAD,Problem View
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
3,0,0,0,0,0

In [70]:
df_unpacked.to_pickle('mtprocessed.p')

In [72]:
df_read = pd.read_pickle('mtprocessed.p')
df_read.head(5)

,Changing axis bounds,Changing axis intervals,Choose Graphical Refl-v,Choose Graphical a,Choose Graphical h,Choose Graphical k,"Convert unit, mixed","Convert unit, multiplier","Convert unit, standard",Correctly placing points,Define Variable,Edit Algebraic a,Edit Algebraic h,Edit Algebraic k,Edit Algebraic refl-v,Entering a computed linear value,Entering a computed quadratic value,Entering a given,Entering a given linear value,Entering a given quadratic value,Entering a point,"Entering slope, GLF","Entering slope, SIF",Entering the slope,Entering the y-intercept,"Entering x-intercept, GLF","Entering x-intercept, SIF","Entering y-intercept, GLF","Entering y-intercept, SIF",Excluding the line when shading,"Find X, Simple","Find X, negative slope","Find X, positive slope","Find Y, Simple","Find Y, any form","Find Y, negative slope","Find Y, positive slope",Identify Parent Curve,Identify Parent Description,Identify Parent Equation,Identifying units,Including the line when shading,Labelling point of intersection,Labelling the axes,"Negative Constant, GLF","Negative Constant, SIF",Placing coordinate point,"Positive Constants, GLF","Positive Constants, SIF",Setting the slope,Setting the y-intercept,Shading GLF equation with negative slope,Shading GLF equation with positive slope,Shading SIF equation with negative slope,Shading SIF equation with positive slope,Shading greater than,Shading less than,Using difficult numbers,Using large numbers,Using simple numbers,Using small numbers,"Write expression, initial and change","Write expression, initial and point","Write expression, negative slope","Write expression, positive slope","Write expression, quadratic","Write expression, ratio","Write expression, two points",SkillRule: Add/Subtract,SkillRule: Apply Exponent,SkillRule: Calculate Eliminate Parens,SkillRule: Calculate negative coefficient,SkillRule: Consolidate vars with coeff,"SkillRule: Consolidate vars, any","SkillRule: Consolidate vars, no coeff",SkillRule: Do Multiply - Whole nested,SkillRule: Done?,SkillRule: Eliminate Parens 1,SkillRule: Eliminate Parens 2,SkillRule: Extract to consolidate vars,SkillRule: Isolate negative,SkillRule: Isolate positive,SkillRule: Make variable positive 1,SkillRule: Make variable positive 2,SkillRule: Multiply/Divide,SkillRule: Remove coefficient,SkillRule: Remove constant,SkillRule: Remove negative coefficient,SkillRule: Remove positive coefficient,SkillRule: Select Combine Terms,SkillRule: Select Eliminate Parens,"SkillRule: Select Multiply/Divide, nested",SkillRule: Select Multiply,SkillRule: Variable in denominator,"SkillRule: ax+b=c, negative",SkillRule: done infinite solutions,SkillRule: done no solutions,SkillRule: invert-mult,combine-like-terms-r-sp,combine-like-terms-sp,combine-like-terms-whole-sp,distribute-sp,factor-quadratic-sp,factor-sp,perform-mult-r-sp,perform-mult-row2-sp,perform-mult-sp,perform-mult-whole-sp,qft-den-sp,qft-num1-sp,qft-num2-sp,simplify-fractions-sp,Correct First Attempt,Sum_OCs,Num_KCs,EG,DEMO,JAN,FEB,TRANSFORM,LIT,SYS,BH,DIST,NEW,L5,L6,DEC,REAL,FOR,1PT,2PT,PROP,L1F,NOV,PERCENT,IPC,CTA,ES,LINEAR,QUAD,Problem View
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
3,0,0,0,0,0